# RCB'S BEST SEASON WAS 2016, THEIR SQUAD WAS 

**https://www.espncricinfo.com/series/ipl-2016-968923/royal-challengers-bangalore-squad-969877/series-squads**

home teams for every team:
IND: I.S. Bindra Punjab Cricket Association Stadium

IND: Sawai Mansingh Stadium, Jaipur

IND: Eden Gardens, Kolkata

IND: Wankhede Stadium, Mumbai

IND: M.Chinnaswamy Stadium, Bengaluru

IND: Narendra Modi Stadium, Motera, Ahmedabad

IND: Arun Jaitley Stadium, Delhi

IND: Rajiv Gandhi International Stadium, Uppal, Hyderabad

IND: Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow

IND: MA Chidambaram Stadium, Chepauk, Chennai

                                                      template

In [4]:
import re
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
options = Options()
options.headless = True
import time

In [1]:
data = [
    {
        #allrounder
        "role": None,
        "strike rate": None,
        "wickets": None,
        "average": None,
        "awards": None,
        "sixes": None,
        "fours": None,
        "economy": None,
        "batting average": None,
        "bowling average": None,
        "bat_style": None,
        "bowl_style": None,
        "importance": None,
    },
    {
        #bowler
        "role": None,
        "strike rate": None,
        "wickets": None,
        "stadium1 economy": None,
        "stadium10 economy": None,
        "handed": None,
        "awards": None,
        "economy": None,
        "bowling average": None,
        "bat_style": None,
        "bowl_style": None,
        "importance": None,
    },
    {
        #batsman
        "role": None,
        "strike rate": None,
        "runs": None,
        "stadium1 average": None,
        "stadium10 strike rate": None,
        "bat_style": None,
        "bowl_style": None,
        "awards": None,
        "sixes": None,
        "fours": None,
        "economy": None,
        "batting average": None,
        "importance": None,
    }
]

In [6]:
class Player:
    def __init__(self,name,age,role):
        self.name = name
        self.age = age   
        self.role = role
squad = {}

finding rosters and cleaning them up

In [ ]:
urls = ['https://www.espncricinfo.com/series/ipl-2016-968923/royal-challengers-bangalore-squad-969877/series-squads']
for url in urls:
    rqsts = requests.get(url)
    soup = BeautifulSoup(rqsts.content,'lxml')
    names = soup.find_all('div', class_ = 'ds-flex ds-flex-row ds-items-center ds-justify-between')
    names = [name.text.strip() for name in names]
    names = [name.replace('(c)', '').replace('†', '') for name in names if "Withdrawn" not in name]
    ages = soup.find_all('span', class_ = 'ds-text-compact-xxs ds-font-bold')
    ages = [age.text.strip() for age in ages]
    ages = [int(re.search(r'(\d+)y', age).group(1)) for age in ages if re.search(r'(\d+)y', age)]
    roles = soup.find_all('p',class_ = 'ds-text-tight-s ds-font-regular ds-mb-2 ds-mt-1')
    roles = [role.text.strip() for role in roles]
    for i in range(len(roles)):
        if 'Batter' in roles[i]:
            roles[i] = 'Batter'
        elif 'Allrounder' in roles[i]:
            roles[i] = 'AllRounder'
        elif 'Bowler' in roles[i]:
            roles[i] = 'Bowler'
    removes = len(names) - len(ages)
    n = removes
    del ages[n:]
    del roles[n:]
    for name, age, role in zip(names, ages, roles):
        squad[name] = Player(name=name,age=age,role=role)

making a function that takes stuff from statsguru

In [3]:
def stats_taking(player,i):
    driver = webdriver.Chrome()
    driver.get("https://stats.espncricinfo.com/ci/engine/stats/index.html#:~:text=Statsguru%20%7C%20Searchable%20Cricket%20Statistics%20database%20%7C%20ESPNcricinfo.com")
    driver.maximize_window()
    search_box = driver.find_element(By.NAME, "search")
    search_query = player.name
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    link = driver.find_element(By.XPATH, "//a[starts-with(text(), 'Players and Officials')]")
    link.click()
    link = driver.find_element(By.XPATH, "//a[text()='Twenty20 matches player']")
    time.sleep(3)
    menu_url = driver.current_url    #this behaves like a main menu for the driver to come back to for finding other statistics using this page
    player_info = driver.find_element(By.XPATH, "//p[@style='padding-bottom:10px']").text
    details = player_info.split('\n')[1]
    details = details.replace('Player details: ','')
    bat_style = details.split(';')[0]                                            #batting style
    bowl_style = details[1]                                                   #bowling style
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='cumulative']")
    radio_button.click()
    time.sleep(0.1)
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='awards_match']")
    radio_button.click()
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
    submit_button.click()
    try:
        player_of_match_elements = driver.find_elements(By.XPATH, "//td[text()='player of the match']")
        potm_awards = len(player_of_match_elements)                         #awards
        driver.get(menu_url)  
    except Exception as e:
        pass
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='results']")
    radio_button.click()
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
    submit_button.click()
    time.sleep(0.1)
    try:
        wins = driver.find_elements(By.XPATH, "//td[text()='won']")
        losses = driver.find_elements(By.XPATH, "//td[text()='lost']")
        importance = len(wins)/len(losses)                                 #importance
        driver.get(menu_url)  
    except Exception as e:
        pass
    if player.role == 'AllRounder':
        radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='bowling']")       
        radio_button.click()
        submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
        submit_button.click()
        time.sleep(0.1)
        row = driver.find_element(By.XPATH, "//tr[@class='data1']")
        cells = row.find_elements(By.TAG_NAME, "td")
        economy = int(cells[9].text)                                  #economy
        wickets = int(cells[6].text)                                  #wickets
        bowling_average = float(cells[8].text)                       #bowling average
        data[i]['wickets'] = wickets
        data[i]['bowling average'] = bowling_average
        data[i]['economy'] = economy
        driver.get(menu_url)
        radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='batting']")
        radio_button.click()
        submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
        submit_button.click()
        time.sleep(3)
        table_row = driver.find_element(By.CSS_SELECTOR, "tr.data1")
        data = table_row.find_elements(By.TAG_NAME, "td")
        runs = int(data[4].text) 
        data[i]['runs'] = runs
        batting_average = float(data[7].text)
        data['batting_average'] = batting_average
        strike_rate = float(data[9].text)
        data[i]['strike_rate'] = strike_rate
        fours = int(data[12].text)
        data[i]['fours'] = fours
        sixes = int(data[13].text) 
        data[i]['sixes'] = sixes
    elif player.role == 'Bowler':
        []
        pass
    elif player.role == 'Batter':
        pass
    
    driver.quit()

In [ ]:
i = 0
for player in list(squad.values()):
    stats_taking(player.name,i = i)
    i += 1

writing into a json file

In [9]:
# Write to a JSON file
with open("my_dataset.json", "w") as f:
    json.dump(data, f, indent=4)  # `indent=4` makes it human-readable